In [4]:
import zipline
# from zipline.examples import buyapple

%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [19]:
%%zipline --start 2014-1-1 --end 2018-1-1 -o data/dma.pickle --capital-base 1000


from zipline.api import order, order_target, record, symbol, order_value
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

# For this example, we're going to write a simple momentum script.  
# When the stock goes up quickly, we're going to buy; 
# when it goes down we're going to sell.  
# Hopefully we'll ride the waves.

# To run an algorithm in Quantopian, you need two functions: 
# initialize and handle_data.
def initialize(context):
    # The initialize function sets any data or variables that 
    # you'll use in your algorithm. 
    # For instance, you'll want to define the security 
    # (or securities) you want to backtest.  
    # You'll also want to define any parameters or values 
    # you're going to use later. 
    # It's only called once at the beginning of your algorithm.
    
    # In our example, we're looking at Apple.  
    # If you re-type this line you'll see 
    # the auto-complete that is available for security. 
    context.security = symbol('AAPL')

    # limit the number of companies returned in a fundamentals query
    context.limit = 10

def before_trading_start(context, data):
    context.fundamentals = get_fundamentals(
        query(
            fundamentals.valuation_ratios.pb_ratio,
            fundamentals.valuation_ratios.pe_ratio,
        )
        .filter(
            fundamentals.valuation_ratios.pe_ratio < 14
        )
        .filter(
            fundamentals.valuation_ratios.pb_ratio < 2
        )
        .order_by(
            fundamentals.valuation.market_cap.desc()
        )
        .limit(context.limit)
    )
    
    update_universe(context.fundamentals.columns.values)

# The handle_data function is where the real work is done.  
# This function is run either every minute 
# (in live trading and minute backtesting mode) 
# or every day (in daily backtesting mode).
def handle_data(context, data):
    # We've built a handful of useful data transforms for you to use,
    # such as moving average. 
    
    # To make market decisions, we're calculating the stock's 
    # moving average for the last 5 days and its current price. 
    average_price = data[context.security].mavg(5)
    current_price = data[context.security].price
    MA1 = data[context.security].mavg(50)
    MA2 = data[context.security].mavg(200)
    current_positions = context.portfolio.positions[context.security].amount
    
    # Another powerful built-in feature of the Quantopian backtester is the
    # portfolio object.  The portfolio object tracks your positions, cash,
    # cost basis of specific holdings, and more.  In this line, we calculate
    # the current amount of cash in our portfolio.   
    cash = context.portfolio.cash
    print("cash before trade %s" % (cash))

    for stock in data:
        current_position = context.portfolio.positions[stock].amount
        stock_price = data[stock].price
        plausible_investment = cash / context.limit
     
        share_amount = int(plausible_investment / stock_price)

        try:
                if stock_price < plausible_investment:
                    if current_position == 0:
                        if context.fundamentals[stock]['pe_ratio'] < 11:
                            order(stock, share_amount)
    
        except Exception as e:
            print(str(e))
    
    # Here is the meat of our algorithm.
    # If the current price is 1% above the 5-day average price 
    # AND we have enough cash, then we will order.
    # If the current price is below the average price, 
    # then we want to close our position to 0 shares.

    # if current_price > 1.01*average_price and cash > current_price:
        
    #     # Need to calculate how many shares we can buy
    #     number_of_shares = int(cash/current_price)
        
    #     # Place the buy order (positive means buy, negative means sell)
    #     order(context.security, +number_of_shares)
    #     print("Buying %s %s" % (number_of_shares, context.security.symbol))
        

    # elif current_price < average_price:
        
    #     # Sell all of our shares by setting the target position to zero
    #     order_target(context.security, 0)
    #     print("Selling %s" % (context.security.symbol))
    
    # You can use the record() method to track any custom signal. 
    # The record graph tracks up to five different variables. 
    # Here we record the Apple stock price.
    record(stock_price=data[context.security].price)

NameError: name 'get_fundamentals' is not defined

In [23]:
from yahoofinancials import YahooFinancials as YF

tick = YF('AAPL')

tick.get_financial_stmts('annual', 'income', True)

{'incomeStatementHistory': {'AAPL': [{'2019-09-28': {'costOfRevenue': 161782000000,
     'discontinuedOperations': None,
     'ebit': 63930000000,
     'effectOfAccountingCharges': None,
     'extraordinaryItems': None,
     'grossProfit': 98392000000,
     'incomeBeforeTax': 65737000000,
     'incomeTaxExpense': 10481000000,
     'interestExpense': -3576000000,
     'minorityInterest': None,
     'netIncome': 55256000000,
     'netIncomeApplicableToCommonShares': 55256000000,
     'netIncomeFromContinuingOps': 55256000000,
     'nonRecurring': None,
     'operatingIncome': 63930000000,
     'otherItems': None,
     'otherOperatingExpenses': None,
     'researchDevelopment': 16217000000,
     'sellingGeneralAdministrative': 18245000000,
     'totalOperatingExpenses': 196244000000,
     'totalOtherIncomeExpenseNet': 1807000000,
     'totalRevenue': 260174000000}},
   {'2018-09-29': {'costOfRevenue': 163756000000,
     'discontinuedOperations': None,
     'ebit': 70898000000,
     'effec

In [34]:
from iexfinance.refdata import get_symbols
from iexfinance.stocks import Stock
import pandas as pd

symbols=get_symbols(output_format='pandas', token="pk_1dd03915b1bd48c8aa96fa6340d2bab1")

In [106]:
symbols['type']='cs'
symbols_only=symbols[['symbol']]

stockList = symbols_only.head(20).values.flatten().tolist()

batch = Stock(stockList, token="pk_1dd03915b1bd48c8aa96fa6340d2bab1")
shares = pd.DataFrame(batch.get_quote()).transpose()

,avgTotalVolume,calculationPrice,change,changePercent,close,closeTime,companyName,delayedPrice,delayedPriceTime,extendedChange,...,openTime,peRatio,previousClose,previousVolume,primaryExchange,symbol,volume,week52High,week52Low,ytdChange
A,2066775,close,-2.55,-0.03168,None,None,"Agilent Technologies, Inc.",None,None,None,...,None,31.82,80.5,2919380,New York Stock Exchange,A,None,90.64,65.35,-0.12418
AA,7053071,close,-0.86,-0.05599,None,None,Alcoa Corp.,None,None,None,...,None,-2.39,15.36,5465689,New York Stock Exchange,AA,None,31.45,13.62,-0.39269
AAAU,85983,close,-0.31,-0.01872,None,None,Perth Mint Physical Gold ETF,None,None,None,...,None,None,16.56,306580,NYSE Arca,AAAU,None,16.8,12.66,0.04758
AACG,24918,close,0.09,0.08571,None,None,ATA Creativity Global,None,None,None,...,None,-2.51,1.05,36145,NASDAQ,AACG,None,4.5,0.955,-0.06989
AADR,12643,close,-1.61,-0.02953,None,None,WCM/BNY Mellon Focused Growth ADR ETF,None,None,None,...,None,None,54.53,8123,NYSE Arca,AADR,None,57.51,44.09,-0.05963
AAL,11428544,close,-2.33,-0.09155,None,None,"American Airlines Group, Inc.",None,None,None,...,None,6.06,25.45,21713199,NASDAQ,AAL,None,36.73,23.05,-0.29375
AAMC,9689,close,2.59,0.09965,None,None,Altisource Asset Management Corp.,None,None,None,...,None,-8.26,25.99,35739,NYSE American,AAMC,None,41.85,7.6,1.42135
AAME,2641,close,-0.13,-0.05138,None,None,Atlantic American Corp.,None,None,None,...,None,-26.14,2.53,698,NASDAQ,AAME,None,2.95,1.46,0.15462
AAN,984357,close,-0.58,-0.0134,None,None,"Aaron's, Inc.",None,None,None,...,None,94.07,43.28,2157153,New York Stock Exchange,AAN,None,78.65,40.79,-0.2696
AAOI,930519,close,-0.39,-0.03683,None,None,"Applied Optoelectronics, Inc.",None,None,None,...,None,-5.18,10.59,725646,NASDAQ,AAOI,None,17.05,8.34,-0.19593


In [99]:
shares

{'A': {'avgTotalVolume': 2066775,
  'calculationPrice': 'close',
  'change': -2.55,
  'changePercent': -0.03168,
  'close': None,
  'closeTime': None,
  'companyName': 'Agilent Technologies, Inc.',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexLastUpdated': 1582664398007,
  'iexMarketPercent': 0.0381308905517068,
  'iexRealtimePrice': 77.94,
  'iexRealtimeSize': 100,
  'iexVolume': 105546,
  'isUSMarketOpen': False,
  'lastTradeTime': 1582664399226,
  'latestPrice': 77.95,
  'latestSource': 'Close',
  'latestTime': 'February 25, 2020',
  'latestUpdate': 1582664637134,
  'latestVolume': None,
  'low': None,
  'marketCap': 24224365600,
  'oddLotDelayedPrice': None,
  'oddLotDelayedPriceTime': None,
  'open': None,
  'openTime': None,
  'peRatio': 31.82,
  'previousClose': 